In [1]:

import subprocess
import os

!source /etc/network_turbo

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
        
import os
# We need Hugging Face の 镜像网站！
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [2]:
from datasets import load_dataset
import time
start_time = time.time()
train_dataset = load_dataset("json", data_files="data/translation2019zh_train.json")
end_time = time.time()
print(f"加载train集耗时: {end_time - start_time:.2f} 秒")
valid_dataset = load_dataset("json", data_files="data/translation2019zh_valid.json")



加载train集耗时: 1.01 秒


In [3]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'chinese'],
        num_rows: 5161434
    })
})

In [4]:
split_datasets = train_dataset["train"].train_test_split(test_size=0.1, seed=42)
split_datasets["train"][1]

{'english': 'My fervor is the decoration in the decadent native place blurred flowers, the surface magnificently with making noise, has hidden too many despairs.',
 'chinese': '我的激情是装饰的花卉，颓废的本土模糊了表面的华丽与喧嚣，隐藏了太多的绝望。'}

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline



model_name = "t5-small"
translator = pipeline("translation", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

max_length = 1024
def preprocess_function(examples):
    inputs = [ex for ex in examples["english"]]
    targets = [ex for ex in examples["chinese"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/317 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

/root/miniconda3/lib/python3.12/site-packages/transformers/pipelines/__init__.py:1244: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(
Device set to use cuda:0


Map:   0%|          | 0/4645290 [00:00<?, ? examples/s]

Map:   0%|          | 0/516144 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import evaluate

metric = evaluate.load("sacrebleu")

In [12]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [17]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="model_dir",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result